# Fewsats Client

This notebook demonstrates the implementation of the Fewsats client, a Python SDK for interacting with the Fewsats API. We'll build the client step by step, explaining each part as we go.

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.utils import *
import os
import requests
from typing import Dict, Any, List

## The Fewsats Client

We'll start by creating the base `Fewsats` class. This class will handle authentication and provide the foundation for our API interactions.

In [ ]:
#| export
class Fewsats:
    def __init__(self, api_key: str = None, base_url: str = "https://hub-5n97k.ondigitalocean.app/"):
        self.api_key = api_key or os.environ.get("HUB_API_TOKEN")
        if not self.api_key:
            raise ValueError("API key not provided and HUB_API_TOKEN environment variable is not set")
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({"Authorization": f"Token {self.api_key}"})

@patch
def _request(self: Fewsats, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
    url = f"{self.base_url}/{endpoint}"
    response = self.session.request(method, url, **kwargs)
    response.raise_for_status()
    return response.json()


In [ ]:
client = Fewsats(api_key="nJ2ro5V9x69JDEhEGyjnzfg6kTkk4LQjukIbfDmuOec",
                 base_url="http://localhost:8000")

Now that we have our base client, let's add methods to interact with the API. We'll use the `@patch` decorator to add these methods to our `Fewsats` class.

## Payment Methods

First, let's add a method to retrieve the user's payment methods. This will be useful for checking which card will be used for purchases.

In [ ]:
#| export
@patch
def get_payment_methods(self: Fewsats) -> List[Dict[str, Any]]:
    """Retrieve the user's payment methods.
    
    Returns:
        List[Dict[str, Any]]: A list of payment methods associated with the user's account.
    """
    return self._request("GET", "v0/stripe/payment-methods")


In [ ]:

payment_methods = client.get_payment_methods()
payment_methods

[{'id': 1,
  'last4': '4242',
  'brand': 'visa',
  'exp_month': 12,
  'exp_year': 2034,
  'is_default': True}]

## Preview Purchase

Now, let's add a method to preview a purchase. This method will use the default payment method if a charge is needed.

In [ ]:
#| export
@patch
def preview_purchase(self: Fewsats, l402_url: str) -> Dict[str, Any]:
    """Preview a purchase for the given L402 URL.
    
    This method will use the default payment method if a charge is needed.
    
    Args:
        l402_url (str): The L402 URL for the purchase.
    
    Returns:
        Dict[str, Any]: The preview details of the purchase.
    """
    return self._request("POST", "v0/l402/preview/purchase", json={"l402_url": l402_url})


In [ ]:

# Let's test our preview_purchase method
l402_url = "https://api.fewsats.com/v0/storage/download/2e54ac29-5945-4b5f-93db-998a535a5a49"
preview = client.preview_purchase(l402_url)
preview

{'invoice': {'description': 'USK House',
  'amount_usd': 1,
  'amount_btc': 17,
  'macaroon': 'AgELZmV3c2F0cy5jb20CQgAAixN/Z/Yq2hOpxABgPnQ2Pp6GZeYbVOF01xAz/yL31noCFMS09vuHHdQXNTkos6dnucW9CK3dDGQqUFpAfCMDIAACLGZpbGVfaWQ9MmU1NGFjMjktNTk0NS00YjVmLTkzZGItOTk4YTUzNWE1YTQ5AAIfZXhwaXJlc19hdD0yMDI1LTA4LTI0VDEwOjQwOjI5WgAABiCw1A4Ve05yJrCKb2IDKWSK8gAHl9JqdbzavDLhr4yMCQ==',
  'invoice': 'lnbc170n1pndq5qapp53vfh7elk9tdp82wyqpsruapk860gve0xrd2wzaxhzqel7ghh6eaqdq024f5kgzgda6hxegcqzzsxqyz5vqsp5z67p2wxkyxm6p594d0dwcx55nn7kk3ruc0qhna7zvtl364g0fsys9qxpqysgq75nauzznx07hxp5sc0kkrwfcgy57qvxrut3wf0mwhmrqh7lgyhcxymq7vm5agfm4tkjmhnn409rnzschmz7hcw5vhwjnxlxm7u680xcq2qp8wl'},
 'transaction': {'current_balance': 1999,
  'balance_to_apply': 1,
  'amount_to_charge': 0,
  'final_balance': 1998},
 'already_purchased': True,
 'purchase': {'id': 3,
  'user_id': 1,
  'l402_url': 'https://api.fewsats.com/v0/storage/download/2e54ac29-5945-4b5f-93db-998a535a5a49',
  'currency': 'usd',
  'amount': 1,
  'macaroon': 'AgELZmV

## Make a Purchase

Finally, let's add a method to make a purchase. Like the preview method, this will use the default payment method if a charge is needed.

In [ ]:
#| export
@patch
def purchase(self: Fewsats, l402_url: str) -> Dict[str, Any]:
    """Make a purchase for the given L402 URL.
    
    This method will use the default payment method if a charge is needed.
    
    Args:
        l402_url (str): The L402 URL for the purchase.
    
    Returns:
        Dict[str, Any]: The details of the completed purchase.
    """
    return self._request("POST", "v0/l402/purchases", json={"l402_url": l402_url})


In [ ]:

# Let's test our purchase method
purchase_result = client.purchase(l402_url)
print(f"Purchase result: {purchase_result}")

Purchase result: {'id': 3, 'created_at': '2024-08-28T14:47:26.904Z', 'l402_url': 'https://api.fewsats.com/v0/storage/download/2e54ac29-5945-4b5f-93db-998a535a5a49', 'macaroon': 'AgELZmV3c2F0cy5jb20CQgAADkZv2VQjgOG+iwoFB5CXg5yInkncy5xEGQZKrkphETmU9+MjeNM4bsj1j5l5KKvildd2pHdtrW+ChXQOQ/fZzgACLGZpbGVfaWQ9MmU1NGFjMjktNTk0NS00YjVmLTkzZGItOTk4YTUzNWE1YTQ5AAIfZXhwaXJlc19hdD0yMDI1LTA4LTIzVDE0OjQ3OjI1WgAABiBIoB5sUXEXijxuyL2cjn0WIgLpOCtt8UcGaGn9N/8n0Q==', 'invoice': 'lnbc170n1pnv7wrapp5perxlk25ywqwr05tpgzs0yyhswwg38jfmn9ec3qeqe92ujnpzyusdq024f5kgzgda6hxegcqzzsxqyz5vqsp54ceypjapjxntxrtnhkyfrds7m5u9stn8x6gkmf5dx6z56ksy6l3q9qxpqysgqnkxdtwqxhkvrav3d3wv83yxn45yrdywxu2mnyh4p7ecncuqsv4gn0xrfl6w57hd9lnc5eghdf536dqy5gf2uuly04nrtdjcwvezkt6cqtttnzj', 'preimage': 'b184b87e7a4f802bf41cd26c82d9d883b9cf7bb86f8dcfc4f394634ff711f628', 'amount': 1, 'currency': 'usd', 'description': 'USK House'}


## Conclusion

We've now built a simple, intuitive Fewsats client that can:
1. Retrieve the user's payment methods
2. Preview a purchase
3. Make a purchase

Both the preview and purchase methods automatically use the default payment method if a charge is needed. This client provides a straightforward way to interact with the Fewsats API, making it easy for developers to integrate Fewsats functionality into their applications.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()